## Tools for building Henderson's Mixed Model Equations

Here we will see how the mixed model equations (HMME) can be built given a data set and a model string. 


In [7]:
include ("../../PedModule.jl/src/PedModule.jl")
using DataFrames

In [58]:
function GaussSeidel(A,x,b;tol=0.000001)
    n = size(x,1)
    for i=1:n
        x[i] = ((b[i] - A[i,:]*x)/A[i,i])[1,1] + x[i]
    end
    diff = sum((A*x-b).^2)
    iter = 0
    while ((diff/n > tol) & (iter<1000))
        iter += 1
        for i=1:n
            x[i] = ((b[i] - A[i,:]*x)/A[i,i])[1,1] + x[i]
        end
        diff = sum((A*x-b).^2)
        println(iter," ",diff/n)
    end
    return x
end

function Jacobi(A,x,b,p;tol=0.000001)
    D       = diag(A)
    res     = A*x
    resid   = b-res
    tempSol = resid./D
    diff    = sum(resid.^2)
    n    = size(A,1)
    iter = 0
    while ((diff/n > tol) & (iter<1000))
        iter += 1
        x = p*tempSol + (1-p)*x
        res     = A*x
        resid   = b-res
        tempSol = resid./D + x
        diff    = sum(resid.^2)
        println(iter," ",diff/n)
    end
    return x
end


type TermStrVal
    str::String
    value::Float64
end

type TermLvlVal
    level::String
    value::Float64
end

type ModelTerm 
    iModel::Int64
    trmStr::String
    nFactors::Int64
    factors::Array{Symbol,1}
    str::Array{String,1}                    # used to store the data for this term as strings
    val::Array{Float64,1}
    startPos::Int64                         # start pos in HMME
    nLevels::Int64                           
    X::SparseMatrixCSC{Float64,Int64}
    names::Array{Any,1}
end

type MME
    modelVec::Array{String,1}
    modelTerms::Array{ModelTerm,1}
    modelTermDict::Dict{String,ModelTerm}
    lhsVec::Array{Symbol,1}
    covVec::Array{Symbol,1}
    pedTrmVec::Array{String,1}
    mmeLhs
    mmeRhs
    ped
    Gi::Array{Float64,2}
    Ri::Array{Float64,2}
    Ai
    mmePos::Int64
end

function mkDict(a)
    aUnique = unique(a)
    d = Dict()
    names = Array(Any,size(aUnique,1))
    for (i,s) in enumerate(aUnique)
        names[i] = s
        d[s] = i
    end
    return d,names
end

function getTerm(trmStr,m)
    trm = ModelTerm(m,trmStr,0,[],[],[],0,0,spzeros(0,0),[])
    factorVec = split(trmStr,"*")
    trm.nFactors = length(factorVec)
    trm.factors = [symbol(strip(f)) for f in factorVec]
    return trm
end

function initMME(models::String,R::Array{Float64,2})
    # returns an MME object for muilding the mme corresponding 
    # to the input string
    if models==""
        println("modelEquation is empty\n")
        return
    end
    modelVec = split(models,[';','\n'],false)
    nModels  = size(modelVec,1)
    lhsVec   = Symbol[]
    modelTerms = ModelTerm[]
    dict = Dict{String,ModelTerm}()
    for (m,model) = enumerate(modelVec)
        lhsRhs = split(model,"=")
        lhsVec = [lhsVec,symbol(strip(lhsRhs[1]))]
        rhs = strip(lhsRhs[2])
        rhsVec = split(rhs,"+")    
        mTrms = [getTerm(strip(trmStr),m) for trmStr in rhsVec]
        modelTerms = [modelTerms, mTrms]
        for (i,trm) = enumerate(modelTerms) # this may need to change
            dict[trm.trmStr] = modelTerms[i]
        end 
    end
    Ri = inv(R)
    return MME(modelVec,modelTerms,dict,lhsVec,[],[],0,0,0,Array(Float64,1,1),Ri,0,1)
end 

function getData(trm::ModelTerm,df::DataFrame,mme::MME)
    nObs = size(df,1)
    trm.str = Array(String,nObs)
    trm.val = Array(Float64,nObs)
    if(trm.factors[1] == :intercept)                     # from Melanie's HW
        str = fill(string(trm.factors[1]),nObs)
        val = fill(1.0,nObs)
    else
        myDf = df[trm.factors]
        if trm.factors[1] in mme.covVec
            str = fill(string(trm.factors[1]),nObs)
            val = df[trm.factors[1]]
        else
            str = [string(i) for i in df[trm.factors[1]]]
            val = fill(1.0,nObs)
        end
        for i=2:trm.nFactors
            if trm.factors[i] in mme.covVec
                str = str .* fill("*"*string(trm.factors[i]),nObs)
                val = val .* df[trm.factors[i]]
            else
                str = str .* fill("*",nObs) .* [string(j) for j in df[trm.factors[i]]]
                val = val .* fill(1.0,nObs)
            end
        end
    end
    trm.str = str
    trm.val = val
end

getFactor1(str) = [strip(i) for i in split(str,"*")][1]

function getX(trm,mme::MME)
    pedSize = 0
    nObs  = size(trm.str,1)
    if trm.trmStr in mme.pedTrmVec
        trm.names   = PedModule.getIDs(mme.ped)
        trm.nLevels = length(mme.ped.idMap)
        xj = int([mme.ped.idMap[getFactor1(i)].seqID for i in trm.str])
    else
        dict,trm.names  = mkDict(trm.str)
        trm.nLevels     = length(dict)
        xj    = int([dict[i] for i in trm.str])
    end
    xi    = (trm.iModel-1)*nObs + [1:nObs]
    xv    = trm.val
    if mme.ped!=0
        pedSize = length(mme.ped.idMap)
        if trm.trmStr in mme.pedTrmVec
            # This is to ensure the X matrix for 
            # additive effect has the correct number of columns
            ii = 1         # adding a zero to
            jj = pedSize   # the last column in row 1
            vv = [0.0]
            xi = [xi,ii]
            xj = [xj,jj]
            xv = [xv,vv]
        end
    end 
    #make sure X has nObs*nModels rows
    nModels = size(mme.lhsVec,1)
    xi = [xi,1,nObs*nModels]
    xj = [xj,1,1]
    xv = [xv,0,0]
    trm.X = sparse(xi,xj,xv)
    trm.startPos = mme.mmePos
    mme.mmePos  += trm.nLevels
end

function getMME(mme::MME, df::DataFrame)
    for trm in mme.modelTerms
        getData(trm,df,mme)
        getX(trm,mme)
    end
    n   = size(mme.modelTerms,1)
    trm = mme.modelTerms[1]
    X   = trm.X
    for i=2:n
        trm = mme.modelTerms[i]
        X = [X trm.X]
    end
    y = df[mme.lhsVec[1]]
    for i=2:size(mme.lhsVec,1)
        y    = [y, df[mme.lhsVec[i]] ] 
    end
    N  = size(y,1)
    ii = 1:N
    jj = fill(1,N)
    vv = y
    ySparse = sparse(ii,jj,vv)
    nObs = size(df,1)
    Ri = kron(mme.Ri,speye(nObs))
    mme.mmeLhs = X'Ri*X
    mme.mmeRhs = X'Ri*ySparse
end

function getNames(mme)
    names = Array(String,0)
    for trm in mme.modelTerms
        for name in trm.names
            push!(names,trm.trmStr*": "*name)
        end
    end
    return names
end  

function covList(mme::MME, covStr::String)
    covVec = split(covStr," ",false) 
    mme.covVec = [symbol(i) for i in covVec]
    nothing
end

function getSolJ(mme::MME, df::DataFrame)
    if size(mme.mmeRhs)==() 
        getMME(mme,df)
    end
    p = size(mme.mmeRhs,1)
    return [getNames(mme) Jacobi(mme.mmeLhs,fill(0.0,p),mme.mmeRhs,0.3,tol=0.000001)]
end

function getSolG(mme::MME, df::DataFrame)
    if size(mme.mmeRhs)==() 
        getMME(mme,df)
    end
    p = size(mme.mmeRhs,1)
    return [getNames(mme) GaussSeidel(mme.mmeLhs,fill(0.0,p),mme.mmeRhs,tol=0.000001)]
end

function setAsRandom(mme::MME,randomStr::String,ped::PedModule.Pedigree, G::Array{Float64,2})
    mme.pedTrmVec = split(randomStr," ",false)
    mme.ped = ped
    mme.Gi = inv(G)
    nothing
end

function addA(mme::MME)
    pedTrmVec = mme.pedTrmVec
    #pedTrm = mme.modelTermDict[mme.pedTrmVec[1]]
    for (i,trmi) = enumerate(pedTrmVec)
        pedTrmi  = mme.modelTermDict[trmi]
        startPosi  = pedTrmi.startPos
        endPosi    = startPosi + pedTrmi.nLevels - 1
        for (j,trmj) = enumerate(pedTrmVec)
            pedTrmj  = mme.modelTermDict[trmj]
            startPosj  = pedTrmj.startPos
            endPosj    = startPosj + pedTrmj.nLevels - 1       
            mme.mmeLhs[startPosi:endPosi,startPosj:endPosj] = 
            mme.mmeLhs[startPosi:endPosi,startPosj:endPosj] + mme.Ai*mme.Gi[i,j] 
        end
    end
end

addA (generic function with 1 method)

In [9]:
ID = ["S1","D1","O1","O3"]
y1 = [100.0, 50.0, 150.0, 40.0]
y2 = [10.0,  12.9, 13.0,  5.0]

4-element Array{Float64,1}:
 10.0
 12.9
 13.0
  5.0

In [10]:
dfMT = DataFrame(Animal=ID,y1=y1,y2=y2)

4x3 DataFrame
| Row | Animal | y1    | y2   |
|-----|--------|-------|------|
| 1   | "S1"   | 100.0 | 10.0 |
| 2   | "D1"   | 50.0  | 12.9 |
| 3   | "O1"   | 150.0 | 13.0 |
| 4   | "O3"   | 40.0  | 5.0  |

In [64]:
models = "y1 = intercept + Animal;
          y2 = intercept + Animal"
R = [10 2; 2 1.0]
mme = initMME(models,R);

In [65]:
getMME(mme,dfMT);

In [66]:
round(full(mme.mmeLhs),2)

10x10 Array{Float64,2}:
  0.67   0.17   0.17   0.17   0.17  -1.33  -0.33  -0.33  -0.33  -0.33
  0.17   0.17   0.0    0.0    0.0   -0.33  -0.33   0.0    0.0    0.0 
  0.17   0.0    0.17   0.0    0.0   -0.33   0.0   -0.33   0.0    0.0 
  0.17   0.0    0.0    0.17   0.0   -0.33   0.0    0.0   -0.33   0.0 
  0.17   0.0    0.0    0.0    0.17  -0.33   0.0    0.0    0.0   -0.33
 -1.33  -0.33  -0.33  -0.33  -0.33   6.67   1.67   1.67   1.67   1.67
 -0.33  -0.33   0.0    0.0    0.0    1.67   1.67   0.0    0.0    0.0 
 -0.33   0.0   -0.33   0.0    0.0    1.67   0.0    1.67   0.0    0.0 
 -0.33   0.0    0.0   -0.33   0.0    1.67   0.0    0.0    1.67   0.0 
 -0.33   0.0    0.0    0.0   -0.33   1.67   0.0    0.0    0.0    1.67